In [1]:
import pandas as pd
import numpy as np

from nltk.corpus import wordnet
import string
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer

import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\himan\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
#Read the CSV file of European Hotel Reviews
csv_data = pd.read_csv('D:\\College\\Sem2\\Dissertation\\Hotel_Reviews.csv')

In [3]:
#Adding the country column by extracting information from the Hotel's Address.
#It was found that the last word of Hotel Address is the Country where hotel is situated. Below is the code for same:
def filter_country(address):
    country = address.split()[-1]
    if (country == 'Kingdom'):
                country = 'United Kingdom'
    return country

csv_data["Country_Name"] = csv_data["Hotel_Address"].apply(lambda x: filter_country(x))

In [4]:
t = csv_data.groupby(['Hotel_Name','Country_Name'])['Positive_Review'].apply('.'.join).reset_index()
e = csv_data.groupby(['Hotel_Name','Country_Name'])['Review_Total_Positive_Word_Counts'].sum().reset_index()
f = csv_data.groupby(['Hotel_Name','Country_Name'])['Reviewer_Score'].mean().reset_index()
t['Review_Total_Positive_Word_Counts'] = e['Review_Total_Positive_Word_Counts']
t['Reviewer_Score'] = f['Reviewer_Score']

In [8]:
t = t.sort_values(by='Hotel_Name', ascending=True)
t

,Hotel_Name,Country_Name,Positive_Review,Review_Total_Positive_Word_Counts,Reviewer_Score
0,11 Cadogan Gardens,United Kingdom,We were particularly impressed by the very wa...,3176,8.845283
1,1K Hotel,France,Location good close to le Marais and 3e arron...,2309,7.861486
2,25hours Hotel beim MuseumsQuartier,Austria,Cool vintage style in the middle of the museu...,15097,8.983309
3,41,United Kingdom,Its central proximity close to all services a...,2606,9.711650
4,45 Park Lane Dorchester Collection,United Kingdom,Everything here are almost perfect the staffs...,323,9.603571
...,...,...,...,...,...
1489,citizenM London Bankside,United Kingdom,No Positive. What an amazing and unique hotel ...,31639,9.112261
1490,citizenM London Shoreditch,United Kingdom,Swift auto check in Good bar and lounge area ...,18118,9.092005
1491,citizenM Tower of London,United Kingdom,The use of technology was impressive UBS outl...,41117,9.135591
1492,every hotel Piccadilly,United Kingdom,The location was the only great aspect of thi...,9335,8.967782


In [9]:
#Finding the reason of Travels : Leisure or Business
#Extracting some important words from the column Tags - here Leisure or Business which are Travel Reasons
dataFrame = csv_data
dataFrame = dataFrame.reset_index(drop=True)
p = [' Leisure trip ',' Business trip ']
dataFrame = dataFrame.reset_index(drop=True)
dataFrame['Leisure'] = dataFrame['Tags'].map(lambda x: 1 if p[0] in x else 0)
dataFrame['Business'] = dataFrame['Tags'].map(lambda x: 2 if p[1] in x else 0)
dataFrame['Travel_Reason'] = dataFrame['Leisure'] + dataFrame['Business']

#Combining the features Leisure or Business extracted from the Tags into one column named Travel_Reason
import random
dataFrame['Travel_Reason'] = dataFrame[dataFrame['Travel_Reason'] == 0]['Travel_Reason'].map(lambda x: 1 if random.random() > 0.2 else 2)
dataFrame['Travel_Reason'] = dataFrame['Travel_Reason'].fillna(0)
dataFrame['Travel_Reason'] = dataFrame['Travel_Reason'] + dataFrame['Business'] + dataFrame['Leisure']
del dataFrame['Leisure'] , dataFrame['Business']

print(dataFrame['Travel_Reason'].value_counts())

#df['Travel_Reason'] = df['Travel_Reason'].replace({1.0: 'Leisure', 2.0: 'Business'})
print(dataFrame['Travel_Reason'].value_counts())

dataFrame['Travel_Reason'] = dataFrame['Travel_Reason'].replace({1:'Leisure', 2:'Business'})


#Finding the travels made in different groups: Solo, Couple, Group, Young Family, Older Family => Travel_Along
#Extracting Couple or Solo or Group or Family with older children or Family with younger Children from Tags in Travel_Along
c = [' Solo traveler ',' Couple ',' Group ',' Family with young children ', ' Family with older children ']
dataFrame['Solo'] = dataFrame['Tags'].map(lambda x: 1 if c[0] in x else 0)
dataFrame['Couple'] = dataFrame['Tags'].map(lambda x: 2 if c[1] in x else 0)
dataFrame['Group'] = dataFrame['Tags'].map(lambda x: 3 if c[2] in x else 0)
dataFrame['Young_Family'] = dataFrame['Tags'].map(lambda x: 4 if c[3] in x else 0)
dataFrame['Older_Family'] = dataFrame['Tags'].map(lambda x: 5 if c[4] in x else 0)
dataFrame['Group_Type'] = dataFrame['Solo'] + dataFrame['Couple'] + dataFrame['Group'] + dataFrame['Young_Family'] + dataFrame['Older_Family']
del dataFrame['Solo'], dataFrame['Couple'],dataFrame['Group'], dataFrame['Young_Family'],dataFrame['Older_Family']

dataFrame['Group_Type'] = dataFrame['Group_Type'].replace({1:'Solo', 2:'Couple',3:'Group', 4:'Young Family',5:'Older Family'})



#Extracting the type of room allocated to the different travellers =>  Room_Type
e = [' Duplex Double Room ',' Duplex Twin Room ',' Large King Room ',' 2 rooms ',' Classic Twin Room ',' Classic Double Room ',' Executive King Room ',' Classic Triple Room ']
dataFrame['Duplex_Double'] = dataFrame['Tags'].map(lambda x: 1 if e[0] in x else 0)
dataFrame['Duplex_Twin'] = dataFrame['Tags'].map(lambda x: 2 if e[1] in x else 0)
dataFrame['Large_King'] = dataFrame['Tags'].map(lambda x: 3 if e[2] in x else 0)
dataFrame['2_rooms'] = dataFrame['Tags'].map(lambda x: 4 if e[3] in x else 0)
dataFrame['Classic_Twin'] = dataFrame['Tags'].map(lambda x: 5 if e[4] in x else 0)
dataFrame['Classic_Double'] = dataFrame['Tags'].map(lambda x: 6 if e[5] in x else 0)
dataFrame['Executive_King'] = dataFrame['Tags'].map(lambda x: 7 if e[6] in x else 0)
dataFrame['Classic_Triple'] = dataFrame['Tags'].map(lambda x: 8 if e[7] in x else 0)

dataFrame['Room_Type'] = dataFrame['Duplex_Double'] + dataFrame['Duplex_Twin'] + dataFrame['Large_King'] + dataFrame['2_rooms'] + dataFrame['Classic_Twin'] + dataFrame['Classic_Double'] + dataFrame['Executive_King'] + dataFrame['Classic_Triple']
del dataFrame['Duplex_Double'], dataFrame['Duplex_Twin'], dataFrame['Large_King'], dataFrame['2_rooms'], dataFrame['Classic_Twin'], dataFrame['Classic_Double'], dataFrame['Executive_King'], dataFrame['Classic_Triple']

print(dataFrame['Room_Type'].value_counts())


#Finding the Number of stays which were made in the respective hotels => Stay_Days
#Extracting the number of Stays from Tags in new column Stay_Days
a = [' Stayed 1 nights ', ' Stayed 2 nights ', ' Stayed 3 nights ', ' Stayed 4 nights ', ' Stayed 5 nights ',' Stayed 6 nights ', ' Stayed 7 nights ', ' Stayed 8 nights ', ' Stayed 9 nights ', ' Stayed 10 nights ', ' Stayed 11 nights ', ' Stayed 12 nights ', ' Stayed 13 nights ',' Stayed 14 nights ', ' Stayed 15 nights ', ' Stayed 16 nights ', ' Stayed 17 nights ', ' Stayed 18 nights ', ' Stayed 19 nights ',' Stayed 20 nights ']
dataFrame['Stayed_1_days']  = dataFrame['Tags'].map(lambda x: 1 if a[0] in x else 0)
dataFrame['Stayed_2_days']  = dataFrame['Tags'].map(lambda x: 2 if a[1] in x else 0)
dataFrame['Stayed_3_days']  = dataFrame['Tags'].map(lambda x: 3 if a[2] in x else 0)
dataFrame['Stayed_4_days']  = dataFrame['Tags'].map(lambda x: 4 if a[3] in x else 0)
dataFrame['Stayed_5_days']  = dataFrame['Tags'].map(lambda x: 5 if a[4] in x else 0)
dataFrame['Stayed_6_days']  = dataFrame['Tags'].map(lambda x: 6 if a[5] in x else 0)
dataFrame['Stayed_7_days']  = dataFrame['Tags'].map(lambda x: 7 if a[6] in x else 0)
dataFrame['Stayed_8_days']  = dataFrame['Tags'].map(lambda x: 8 if a[7] in x else 0)
dataFrame['Stayed_9_days']  = dataFrame['Tags'].map(lambda x: 9 if a[8] in x else 0)
dataFrame['Stayed_10_days'] = dataFrame['Tags'].map(lambda x: 10 if a[9] in x else 0)
dataFrame['Stayed_11_days'] = dataFrame['Tags'].map(lambda x: 11 if a[10] in x else 0)
dataFrame['Stayed_12_days'] = dataFrame['Tags'].map(lambda x: 12 if a[11] in x else 0)
dataFrame['Stayed_13_days'] = dataFrame['Tags'].map(lambda x: 13 if a[12] in x else 0)
dataFrame['Stayed_14_days'] = dataFrame['Tags'].map(lambda x: 14 if a[13] in x else 0)
dataFrame['Stayed_15_days'] = dataFrame['Tags'].map(lambda x: 15 if a[14] in x else 0)
dataFrame['Stayed_16_days'] = dataFrame['Tags'].map(lambda x: 16 if a[15] in x else 0)
dataFrame['Stayed_17_days'] = dataFrame['Tags'].map(lambda x: 17 if a[16] in x else 0)
dataFrame['Stayed_18_days'] = dataFrame['Tags'].map(lambda x: 18 if a[17] in x else 0)
dataFrame['Stayed_19_days'] = dataFrame['Tags'].map(lambda x: 19 if a[18] in x else 0)
dataFrame['Stayed_20_days'] = dataFrame['Tags'].map(lambda x: 20 if a[19] in x else 0)

dataFrame['Stay_Days'] = dataFrame['Stayed_1_days'] + dataFrame['Stayed_2_days'] + dataFrame['Stayed_3_days'] + dataFrame['Stayed_4_days'] + dataFrame['Stayed_5_days'] + dataFrame['Stayed_6_days'] + dataFrame['Stayed_7_days'] + dataFrame['Stayed_8_days'] + dataFrame['Stayed_9_days'] + dataFrame['Stayed_10_days'] + dataFrame['Stayed_11_days'] + dataFrame['Stayed_12_days'] + dataFrame['Stayed_13_days'] + dataFrame['Stayed_14_days'] + dataFrame['Stayed_15_days'] + dataFrame['Stayed_16_days'] + dataFrame['Stayed_17_days'] + dataFrame['Stayed_18_days'] + dataFrame['Stayed_19_days'] + dataFrame['Stayed_20_days']
del dataFrame['Stayed_1_days'] , dataFrame['Stayed_2_days'] , dataFrame['Stayed_3_days'] , dataFrame['Stayed_4_days'], dataFrame['Stayed_5_days'] , dataFrame['Stayed_6_days'] , dataFrame['Stayed_7_days'] , dataFrame['Stayed_8_days'] , dataFrame['Stayed_9_days'] , dataFrame['Stayed_10_days'] , dataFrame['Stayed_11_days'] , dataFrame['Stayed_12_days'] , dataFrame['Stayed_13_days'] , dataFrame['Stayed_14_days'] , dataFrame['Stayed_15_days'] , dataFrame['Stayed_16_days'] , dataFrame['Stayed_17_days'] , dataFrame['Stayed_18_days'] , dataFrame['Stayed_19_days'] , dataFrame['Stayed_20_days']

1.0    429803
2.0     85935
Name: Travel_Reason, dtype: int64
0    478955
6     17560
4     12393
7      3608
5      2436
8       365
1       311
3        71
2        39
Name: Room_Type, dtype: int64


In [10]:
df_final = pd.DataFrame()
df_final[['Hotel_Name','Travel_Reason','Stay_Days','Group_Type','Country_Name','Customer_Nationality']] = dataFrame[['Hotel_Name','Travel_Reason','Stay_Days','Group_Type','Country_Name','Reviewer_Nationality']]

In [11]:
df_final1 = df_final
import re 
def count_travel_reason(word,df_final1):
    return df_final1["Travel_Reason"].str.count(word, re.I) 

df_final1 = df_final1.groupby(['Hotel_Name','Country_Name','Customer_Nationality'])['Travel_Reason'].apply(','.join).reset_index()
df_final1["countLeisure"] = count_travel_reason("Leisure",df_final1)
df_final1["countBusiness"] = count_travel_reason("Business",df_final1)

df_final1['Travel_Reason_x'] = df_final1.apply(lambda x : "Leisure" if x['countLeisure'] >= x['countBusiness']  else "Business", axis=1)


import re 
def count_group(word,df_final2):
    return df_final2["Group_Type"].str.count(word, re.I) 

df_final = df_final.astype(str)
df_final['Group_Type'].replace(['0', '0.0'], '', inplace=True)
print(df_final.Group_Type.value_counts())
df_final2 = df_final.groupby(['Hotel_Name'])['Group_Type'].apply(','.join).reset_index()
df_final2.head(1)

df_final2["countCouple"] = count_group("Couple",df_final2)
df_final2["countSolo"] = count_group("Solo",df_final2)
df_final2["countGroup"] = count_group("Group",df_final2)
df_final2["countYoung"] = count_group("Young Family",df_final2)
df_final2["countOlder"] = count_group("Older Family",df_final2)
df_final2.head(2)




Couple          252294
Solo            108545
Group            65392
Young Family     61015
Older Family     26349
                  2143
Name: Group_Type, dtype: int64


,Hotel_Name,Group_Type,countCouple,countSolo,countGroup,countYoung,countOlder
0,11 Cadogan Gardens,"Couple,Couple,Solo,Couple,Couple,Group,Solo,Co...",90,37,13,9,10
1,1K Hotel,"Couple,Solo,Couple,Group,Solo,Couple,Couple,Yo...",70,49,17,11,0


In [12]:
import numpy as np
df_final2['Group_x'] = df_final2.apply(lambda x : "Couple" if (x['countCouple'] >= x['countSolo']) and (x['countCouple'] >= x['countGroup']) and (x['countCouple'] >= x['countYoung']) and (x['countCouple'] >= x['countOlder']) 
                                       else ("Solo" if (x['countSolo'] >= x['countCouple']) and (x['countSolo'] >= x['countGroup']) and (x['countSolo'] >= x['countYoung']) and (x['countSolo'] >= x['countOlder']) 
                                             else ("Group" if (x['countGroup'] >= x['countCouple']) and (x['countGroup'] >= x['countSolo']) and (x['countGroup'] >= x['countYoung']) and (x['countGroup'] >= x['countOlder']) 
                                                   else ( "Young Family" if (x['countYoung'] >= x['countCouple']) and (x['countYoung'] >= x['countSolo']) and (x['countYoung'] >= x['countGroup']) and (x['countYoung'] >= x['countOlder'])else "Older Family") )), 
                                       axis=1)
df_final2['Group_x'].value_counts()



import re 
def count_group(word,df_final4):
    return df_final4["Stay_Days"].str.count(word, re.I) 

df_final = df_final.astype(str)
df_final['Stay_Days'].replace(['0', '0.0'], '', inplace=True)
print(df_final.Stay_Days.value_counts())
df_final4 = df_final.groupby(['Hotel_Name'])['Stay_Days'].apply(','.join).reset_index()
df_final4.head(1)

df_final4["count1Night"] = count_group("Stayed 1 night",df_final4)
df_final4["count2Night"] = count_group("Stayed 2 night",df_final4)
df_final4["count3Night"] = count_group("Stayed 3 night",df_final4)
df_final4["count4Night"] = count_group("Stayed 4 night",df_final4)
df_final4["count5Night"] = count_group("Stayed 5 night",df_final4)
df_final4["count6Night"] = count_group("Stayed 6 night",df_final4)
df_final4["count7Night"] = count_group("Stayed 7 night",df_final4)
df_final4["count8Night"] = count_group("Stayed 8 night",df_final4)
df_final4["count9Night"] = count_group("Stayed 9 night",df_final4)
df_final4["count10Night"] = count_group("Stayed 10 night",df_final4)
df_final4["count11Night"] = count_group("Stayed 11 night",df_final4)
df_final4["count12Night"] = count_group("Stayed 12 night",df_final4)
df_final4["count13Night"] = count_group("Stayed 13 night",df_final4)
df_final4["count14Night"] = count_group("Stayed 14 night",df_final4)
df_final4["count15Night"] = count_group("Stayed 15 night",df_final4)
df_final4["count16Night"] = count_group("Stayed 16 night",df_final4)
df_final4["count17Night"] = count_group("Stayed 17 night",df_final4)
df_final4["count18Night"] = count_group("Stayed 18 night",df_final4)
df_final4["count19Night"] = count_group("Stayed 19 night",df_final4)
df_final4["count20Night"] = count_group("Stayed 20 night",df_final4)

df_final4.head(2)




df_final4['Stayed_x'] = df_final4.apply(lambda x : "Stayed 1 night" if (x['count1Night'] >= x['count2Night']) and (x['count1Night'] >= x['count3Night']) and (x['count1Night'] >= x['count4Night']) and (x['count1Night'] >= x['count5Night']) and (x['count1Night'] >= x['count6Night']) and (x['count1Night'] >= x['count7Night']) and (x['count1Night'] >= x['count8Night']) and (x['count1Night'] >= x['count9Night']) and (x['count1Night'] >= x['count10Night']) and (x['count1Night'] >= x['count11Night']) and (x['count1Night'] >= x['count12Night']) and (x['count1Night'] >= x['count13Night']) and (x['count1Night'] >= x['count14Night']) and (x['count1Night'] >= x['count15Night']) and (x['count1Night'] >= x['count16Night']) and (x['count1Night'] >= x['count17Night']) and (x['count1Night'] >= x['count18Night']) and (x['count1Night'] >= x['count19Night']) and (x['count1Night'] >= x['count20Night'])
                                       else ("Stayed 2 night" if (x['count2Night'] >= x['count1Night']) and (x['count2Night'] >= x['count3Night']) and (x['count2Night'] >= x['count4Night']) and (x['count2Night'] >= x['count5Night']) and (x['count2Night'] >= x['count6Night']) and (x['count2Night'] >= x['count7Night']) and (x['count2Night'] >= x['count8Night']) and (x['count2Night'] >= x['count9Night']) and (x['count2Night'] >= x['count10Night']) and (x['count2Night'] >= x['count11Night']) and (x['count2Night'] >= x['count12Night']) and (x['count2Night'] >= x['count13Night']) and (x['count2Night'] >= x['count14Night']) and (x['count2Night'] >= x['count15Night']) and (x['count2Night'] >= x['count16Night']) and (x['count2Night'] >= x['count17Night']) and (x['count2Night'] >= x['count18Night']) and (x['count1Night'] >= x['count19Night']) and (x['count2Night'] >= x['count20Night'])
                                        else ("Stayed 3 night" if (x['count3Night'] >= x['count1Night']) and (x['count3Night'] >= x['count2Night']) and (x['count3Night'] >= x['count4Night']) and (x['count3Night'] >= x['count5Night']) and (x['count3Night'] >= x['count6Night']) and (x['count3Night'] >= x['count7Night']) and (x['count3Night'] >= x['count8Night']) and (x['count3Night'] >= x['count9Night']) and (x['count3Night'] >= x['count10Night']) and (x['count3Night'] >= x['count11Night']) and (x['count3Night'] >= x['count12Night']) and (x['count3Night'] >= x['count13Night']) and (x['count3Night'] >= x['count14Night']) and (x['count3Night'] >= x['count15Night']) and (x['count3Night'] >= x['count16Night']) and (x['count3Night'] >= x['count17Night']) and (x['count3Night'] >= x['count18Night']) and (x['count3Night'] >= x['count19Night']) and (x['count3Night'] >= x['count20Night']) 
                                        else ("Stayed 4 night" if (x['count4Night'] >= x['count2Night']) and (x['count4Night'] >= x['count3Night']) and (x['count4Night'] >= x['count1Night']) and (x['count4Night'] >= x['count5Night']) and (x['count4Night'] >= x['count6Night']) and (x['count4Night'] >= x['count7Night']) and (x['count4Night'] >= x['count8Night']) and (x['count4Night'] >= x['count9Night']) and (x['count4Night'] >= x['count10Night']) and (x['count4Night'] >= x['count11Night']) and (x['count4Night'] >= x['count12Night']) and (x['count4Night'] >= x['count13Night']) and (x['count4Night'] >= x['count14Night']) and (x['count4Night'] >= x['count15Night']) and (x['count4Night'] >= x['count16Night']) and (x['count4Night'] >= x['count17Night']) and (x['count4Night'] >= x['count18Night']) and (x['count4Night'] >= x['count19Night']) and (x['count4Night'] >= x['count20Night']) 
                                        else ("Stayed 5 night" if (x['count5Night'] >= x['count2Night']) and (x['count5Night'] >= x['count3Night']) and (x['count5Night'] >= x['count4Night']) and (x['count5Night'] >= x['count1Night']) and (x['count5Night'] >= x['count6Night']) and (x['count5Night'] >= x['count7Night']) and (x['count5Night'] >= x['count8Night']) and (x['count5Night'] >= x['count9Night']) and (x['count5Night'] >= x['count10Night']) and (x['count5Night'] >= x['count11Night']) and (x['count5Night'] >= x['count12Night']) and (x['count5Night'] >= x['count13Night']) and (x['count5Night'] >= x['count14Night']) and (x['count5Night'] >= x['count15Night']) and (x['count5Night'] >= x['count16Night']) and (x['count1Night'] >= x['count17Night']) and (x['count5Night'] >= x['count18Night']) and (x['count5Night'] >= x['count19Night']) and (x['count5Night'] >= x['count20Night']) 
                                        else ("Stayed 6 night" if (x['count6Night'] >= x['count2Night']) and (x['count6Night'] >= x['count3Night']) and (x['count6Night'] >= x['count4Night']) and (x['count6Night'] >= x['count5Night']) and (x['count6Night'] >= x['count1Night']) and (x['count6Night'] >= x['count7Night']) and (x['count6Night'] >= x['count8Night']) and (x['count6Night'] >= x['count9Night']) and (x['count6Night'] >= x['count10Night']) and (x['count6Night'] >= x['count11Night']) and (x['count6Night'] >= x['count12Night']) and (x['count6Night'] >= x['count13Night']) and (x['count6Night'] >= x['count14Night']) and (x['count6Night'] >= x['count15Night']) and (x['count6Night'] >= x['count16Night']) and (x['count6Night'] >= x['count17Night']) and (x['count6Night'] >= x['count18Night']) and (x['count6Night'] >= x['count19Night']) and (x['count6Night'] >= x['count20Night']) 
                                        else ("Stayed 7 night" if (x['count7Night'] >= x['count2Night']) and (x['count7Night'] >= x['count3Night']) and (x['count7Night'] >= x['count4Night']) and (x['count7Night'] >= x['count5Night']) and (x['count7Night'] >= x['count6Night']) and (x['count7Night'] >= x['count1Night']) and (x['count7Night'] >= x['count8Night']) and (x['count7Night'] >= x['count9Night']) and (x['count7Night'] >= x['count10Night']) and (x['count7Night'] >= x['count11Night']) and (x['count7Night'] >= x['count12Night']) and (x['count7Night'] >= x['count13Night']) and (x['count7Night'] >= x['count14Night']) and (x['count7Night'] >= x['count15Night']) and (x['count7Night'] >= x['count16Night']) and (x['count7Night'] >= x['count17Night']) and (x['count7Night'] >= x['count18Night']) and (x['count7Night'] >= x['count19Night']) and (x['count7Night'] >= x['count20Night'])
                                        else ("Stayed 8 night" if (x['count8Night'] >= x['count2Night']) and (x['count8Night'] >= x['count3Night']) and (x['count8Night'] >= x['count4Night']) and (x['count8Night'] >= x['count5Night']) and (x['count8Night'] >= x['count6Night']) and (x['count8Night'] >= x['count7Night']) and (x['count8Night'] >= x['count8Night']) and (x['count8Night'] >= x['count9Night']) and (x['count8Night'] >= x['count10Night']) and (x['count8Night'] >= x['count11Night']) and (x['count8Night'] >= x['count12Night']) and (x['count8Night'] >= x['count13Night']) and (x['count8Night'] >= x['count14Night']) and (x['count8Night'] >= x['count15Night']) and (x['count8Night'] >= x['count16Night']) and (x['count8Night'] >= x['count17Night']) and (x['count8Night'] >= x['count18Night']) and (x['count8Night'] >= x['count19Night']) and (x['count8Night'] >= x['count20Night']) 
                                        else ("Stayed 9 night" if (x['count9Night'] >= x['count2Night']) and (x['count9Night'] >= x['count3Night']) and (x['count9Night'] >= x['count4Night']) and (x['count9Night'] >= x['count5Night']) and (x['count9Night'] >= x['count6Night']) and (x['count9Night'] >= x['count7Night']) and (x['count9Night'] >= x['count8Night']) and (x['count9Night'] >= x['count1Night']) and (x['count9Night'] >= x['count10Night']) and (x['count9Night'] >= x['count11Night']) and (x['count9Night'] >= x['count12Night']) and (x['count9Night'] >= x['count13Night']) and (x['count9Night'] >= x['count14Night']) and (x['count9Night'] >= x['count15Night']) and (x['count9Night'] >= x['count16Night']) and (x['count9Night'] >= x['count17Night']) and (x['count9Night'] >= x['count18Night']) and (x['count9Night'] >= x['count19Night']) and (x['count9Night'] >= x['count20Night'])
                                        else ("Stayed 10 night" if (x['count10Night'] >= x['count2Night']) and (x['count10Night'] >= x['count3Night']) and (x['count10Night'] >= x['count4Night']) and (x['count10Night'] >= x['count5Night']) and (x['count10Night'] >= x['count6Night']) and (x['count10Night'] >= x['count7Night']) and (x['count10Night'] >= x['count8Night']) and (x['count10Night'] >= x['count9Night']) and (x['count10Night'] >= x['count10Night']) and (x['count10Night'] >= x['count11Night']) and (x['count10Night'] >= x['count12Night']) and (x['count10Night'] >= x['count13Night']) and (x['count10Night'] >= x['count14Night']) and (x['count10Night'] >= x['count15Night']) and (x['count10Night'] >= x['count16Night']) and (x['count10Night'] >= x['count17Night']) and (x['count10Night'] >= x['count18Night']) and (x['count10Night'] >= x['count19Night']) and (x['count10Night'] >= x['count20Night'])
                                        else ("Stayed 11 night" if (x['count11Night'] >= x['count2Night']) and (x['count11Night'] >= x['count3Night']) and (x['count11Night'] >= x['count4Night']) and (x['count11Night'] >= x['count5Night']) and (x['count11Night'] >= x['count6Night']) and (x['count11Night'] >= x['count7Night']) and (x['count11Night'] >= x['count8Night']) and (x['count11Night'] >= x['count9Night']) and (x['count11Night'] >= x['count10Night']) and (x['count11Night'] >= x['count1Night']) and (x['count11Night'] >= x['count12Night']) and (x['count11Night'] >= x['count13Night']) and (x['count11Night'] >= x['count14Night']) and (x['count11Night'] >= x['count15Night']) and (x['count11Night'] >= x['count16Night']) and (x['count11Night'] >= x['count17Night']) and (x['count11Night'] >= x['count18Night']) and (x['count11Night'] >= x['count19Night']) and (x['count11Night'] >= x['count20Night'])
                                        else ("Stayed 12 night" if (x['count12Night'] >= x['count2Night']) and (x['count12Night'] >= x['count3Night']) and (x['count12Night'] >= x['count4Night']) and (x['count12Night'] >= x['count5Night']) and (x['count12Night'] >= x['count6Night']) and (x['count12Night'] >= x['count7Night']) and (x['count12Night'] >= x['count8Night']) and (x['count12Night'] >= x['count9Night']) and (x['count12Night'] >= x['count10Night']) and (x['count12Night'] >= x['count11Night']) and (x['count12Night'] >= x['count1Night']) and (x['count12Night'] >= x['count13Night']) and (x['count12Night'] >= x['count14Night']) and (x['count12Night'] >= x['count15Night']) and (x['count12Night'] >= x['count16Night']) and (x['count12Night'] >= x['count17Night']) and (x['count12Night'] >= x['count18Night']) and (x['count12Night'] >= x['count19Night']) and (x['count12Night'] >= x['count20Night'])
                                        else ("Stayed 13 night" if (x['count14Night'] >= x['count2Night']) and (x['count14Night'] >= x['count3Night']) and (x['count14Night'] >= x['count4Night']) and (x['count14Night'] >= x['count5Night']) and (x['count14Night'] >= x['count6Night']) and (x['count14Night'] >= x['count7Night']) and (x['count14Night'] >= x['count8Night']) and (x['count14Night'] >= x['count9Night']) and (x['count14Night'] >= x['count10Night']) and (x['count14Night'] >= x['count11Night']) and (x['count14Night'] >= x['count12Night']) and (x['count14Night'] >= x['count1Night']) and (x['count14Night'] >= x['count14Night']) and (x['count1Night'] >= x['count15Night']) and (x['count14Night'] >= x['count16Night']) and (x['count14Night'] >= x['count17Night']) and (x['count14Night'] >= x['count18Night']) and (x['count14Night'] >= x['count19Night']) and (x['count14Night'] >= x['count20Night'])
                                        else ("Stayed 14 night" if (x['count15Night'] >= x['count2Night']) and (x['count15Night'] >= x['count3Night']) and (x['count15Night'] >= x['count4Night']) and (x['count15Night'] >= x['count5Night']) and (x['count15Night'] >= x['count6Night']) and (x['count15Night'] >= x['count7Night']) and (x['count15Night'] >= x['count8Night']) and (x['count15Night'] >= x['count9Night']) and (x['count15Night'] >= x['count10Night']) and (x['count15Night'] >= x['count11Night']) and (x['count15Night'] >= x['count12Night']) and (x['count15Night'] >= x['count1Night']) and (x['count15Night'] >= x['count14Night']) and (x['count15Night'] >= x['count1Night']) and (x['count15Night'] >= x['count16Night']) and (x['count15Night'] >= x['count17Night']) and (x['count15Night'] >= x['count18Night']) and (x['count15Night'] >= x['count19Night']) and (x['count15Night'] >= x['count20Night'])
                                        else ("Stayed 15 night" if (x['count16Night'] >= x['count2Night']) and (x['count16Night'] >= x['count3Night']) and (x['count16Night'] >= x['count4Night']) and (x['count16Night'] >= x['count5Night']) and (x['count16Night'] >= x['count6Night']) and (x['count16Night'] >= x['count7Night']) and (x['count16Night'] >= x['count8Night']) and (x['count16Night'] >= x['count9Night']) and (x['count16Night'] >= x['count10Night']) and (x['count16Night'] >= x['count11Night']) and (x['count16Night'] >= x['count12Night']) and (x['count16Night'] >= x['count1Night']) and (x['count16Night'] >= x['count14Night']) and (x['count16Night'] >= x['count15Night']) and (x['count16Night'] >= x['count1Night']) and (x['count16Night'] >= x['count17Night']) and (x['count16Night'] >= x['count18Night']) and (x['count16Night'] >= x['count19Night']) and (x['count16Night'] >= x['count20Night'])
                                        else ("Stayed 16 night" if (x['count17Night'] >= x['count2Night']) and (x['count17Night'] >= x['count3Night']) and (x['count17Night'] >= x['count4Night']) and (x['count17Night'] >= x['count5Night']) and (x['count17Night'] >= x['count6Night']) and (x['count17Night'] >= x['count7Night']) and (x['count17Night'] >= x['count8Night']) and (x['count17Night'] >= x['count9Night']) and (x['count17Night'] >= x['count10Night']) and (x['count17Night'] >= x['count11Night']) and (x['count17Night'] >= x['count12Night']) and (x['count17Night'] >= x['count1Night']) and (x['count17Night'] >= x['count14Night']) and (x['count17Night'] >= x['count15Night']) and (x['count17Night'] >= x['count16Night']) and (x['count17Night'] >= x['count1Night']) and (x['count17Night'] >= x['count18Night']) and (x['count17Night'] >= x['count19Night']) and (x['count17Night'] >= x['count20Night'])
                                        else ("Stayed 17 night" if (x['count18Night'] >= x['count2Night']) and (x['count18Night'] >= x['count3Night']) and (x['count18Night'] >= x['count4Night']) and (x['count18Night'] >= x['count5Night']) and (x['count18Night'] >= x['count6Night']) and (x['count18Night'] >= x['count7Night']) and (x['count18Night'] >= x['count8Night']) and (x['count18Night'] >= x['count9Night']) and (x['count18Night'] >= x['count10Night']) and (x['count18Night'] >= x['count11Night']) and (x['count18Night'] >= x['count12Night']) and (x['count18Night'] >= x['count1Night']) and (x['count18Night'] >= x['count14Night']) and (x['count18Night'] >= x['count15Night']) and (x['count18Night'] >= x['count16Night']) and (x['count18Night'] >= x['count17Night']) and (x['count18Night'] >= x['count1Night']) and (x['count18Night'] >= x['count19Night']) and (x['count18Night'] >= x['count20Night'])
                                        else ("Stayed 18 night" if (x['count19Night'] >= x['count2Night']) and (x['count19Night'] >= x['count3Night']) and (x['count19Night'] >= x['count4Night']) and (x['count19Night'] >= x['count5Night']) and (x['count19Night'] >= x['count6Night']) and (x['count19Night'] >= x['count7Night']) and (x['count19Night'] >= x['count8Night']) and (x['count19Night'] >= x['count9Night']) and (x['count19Night'] >= x['count10Night']) and (x['count19Night'] >= x['count11Night']) and (x['count19Night'] >= x['count12Night']) and (x['count19Night'] >= x['count1Night']) and (x['count19Night'] >= x['count14Night']) and (x['count19Night'] >= x['count15Night']) and (x['count19Night'] >= x['count16Night']) and (x['count19Night'] >= x['count17Night']) and (x['count19Night'] >= x['count1Night']) and (x['count19Night'] >= x['count1Night']) and (x['count19Night'] >= x['count20Night'])
                                        else ("Stayed 19 night" if (x['count20Night'] >= x['count2Night']) and (x['count20Night'] >= x['count3Night']) and (x['count20Night'] >= x['count4Night']) and (x['count20Night'] >= x['count5Night']) and (x['count20Night'] >= x['count6Night']) and (x['count20Night'] >= x['count7Night']) and (x['count20Night'] >= x['count8Night']) and (x['count20Night'] >= x['count9Night']) and (x['count20Night'] >= x['count10Night']) and (x['count20Night'] >= x['count11Night']) and (x['count20Night'] >= x['count12Night']) and (x['count20Night'] >= x['count1Night']) and (x['count20Night'] >= x['count14Night']) and (x['count20Night'] >= x['count15Night']) and (x['count20Night'] >= x['count16Night']) and (x['count20Night'] >= x['count17Night']) and (x['count20Night'] >= x['count1Night']) and (x['count20Night'] >= x['count19Night']) and (x['count20Night'] >= x['count1Night'])
                                        else "Stayed 20 night")))))))))))))))))),axis=1)



#df_finalx = df_final1['Hotel_Name'] + df_final1['Travel_Reason'] + df_final2['Group_Type'] + df_final4['Stayed_x']
df_finalx = pd.DataFrame()
df_finalx['Hotel_Name'] = df_final1['Hotel_Name'] 
df_finalx['Country_Name'] = df_final1['Country_Name'] 
df_finalx['Customer_Nationality'] = df_final1['Customer_Nationality'] 
df_finalx['Travel_Reason_x'] = df_final1['Travel_Reason_x']
df_finalx['Group_x'] = df_final2['Group_x']
df_finalx['Stayed_x'] = df_final4['Stayed_x']

#type(df_finalx['Country_Name'])
df_finalx['Stayed_x'] = df_final['Stay_Days'].apply(str)
df_finalx['Travel_Reason_x'] = df_final['Travel_Reason'].apply(str)
df_finalx['Group_x'] = df_final['Group_Type'].apply(str)
  
# print('After join') 
df_finalx['Tags'] = df_finalx[['Travel_Reason_x','Stayed_x','Group_x','Country_Name','Customer_Nationality']].apply(lambda x: ' '.join(x), axis = 1) 

p = df_finalx.drop_duplicates('Hotel_Name')
p.reset_index(drop=True)
p.head(5)

Stayed 1 night     193645
Stayed 2 night     133937
Stayed 3 night      95821
Stayed 4 night      47817
Stayed 5 night      20845
Stayed 6 night       9776
Stayed 7 night       7399
Stayed 8 night       2502
Stayed 9 night       1293
Stayed 10 night       889
Stayed 11 night       418
Stayed 12 night       316
                      290
Stayed 14 night       247
Stayed 13 night       246
Stayed 15 night       117
Stayed 16 night        57
Stayed 17 night        42
Stayed 18 night        33
Stayed 19 night        29
Stayed 20 night        19
Name: Stay_Days, dtype: int64


,Hotel_Name,Country_Name,Customer_Nationality,Travel_Reason_x,Group_x,Stayed_x,Tags
0,11 Cadogan Gardens,United Kingdom,Australia,Leisure,Couple,Stayed 6 night,Leisure Stayed 6 night Couple United Kingdom ...
36,1K Hotel,France,Australia,Business,Group,Stayed 1 night,Business Stayed 1 night Group France Australia
75,25hours Hotel beim MuseumsQuartier,Austria,,Leisure,Couple,Stayed 3 night,Leisure Stayed 3 night Couple Austria
152,41,United Kingdom,,Leisure,Couple,Stayed 4 night,Leisure Stayed 4 night Couple United Kingdom
175,45 Park Lane Dorchester Collection,United Kingdom,Canada,Leisure,Couple,Stayed 5 night,Leisure Stayed 5 night Couple United Kingdom ...


In [13]:
g = p
p = p.drop(['Country_Name','Customer_Nationality','Travel_Reason_x','Group_x','Stayed_x'], axis=1)

p = p.sort_values(by='Hotel_Name', ascending=True)

In [14]:
p

,Hotel_Name,Tags
0,11 Cadogan Gardens,Leisure Stayed 6 night Couple United Kingdom ...
36,1K Hotel,Business Stayed 1 night Group France Australia
75,25hours Hotel beim MuseumsQuartier,Leisure Stayed 3 night Couple Austria
152,41,Leisure Stayed 4 night Couple United Kingdom
175,45 Park Lane Dorchester Collection,Leisure Stayed 5 night Couple United Kingdom ...
...,...,...
64867,citizenM London Bankside,Leisure Stayed 2 night Solo United Kingdom
64939,citizenM London Shoreditch,Leisure Stayed 1 night Couple United Kingdom
65001,citizenM Tower of London,Business Stayed 1 night Solo United Kingdom
65084,every hotel Piccadilly,Business Stayed 1 night Older Family United Ki...


In [15]:
p.head(5)

,Hotel_Name,Tags
0,11 Cadogan Gardens,Leisure Stayed 6 night Couple United Kingdom ...
36,1K Hotel,Business Stayed 1 night Group France Australia
75,25hours Hotel beim MuseumsQuartier,Leisure Stayed 3 night Couple Austria
152,41,Leisure Stayed 4 night Couple United Kingdom
175,45 Park Lane Dorchester Collection,Leisure Stayed 5 night Couple United Kingdom ...


In [16]:
p = p.reset_index(drop=True)
t = t.reset_index(drop=True)

In [17]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
p.head(5)

,Hotel_Name,Tags
0,11 Cadogan Gardens,Leisure Stayed 6 night Couple United Kingdom ...
1,1K Hotel,Business Stayed 1 night Group France Australia
2,25hours Hotel beim MuseumsQuartier,Leisure Stayed 3 night Couple Austria
3,41,Leisure Stayed 4 night Couple United Kingdom
4,45 Park Lane Dorchester Collection,Leisure Stayed 5 night Couple United Kingdom ...


In [18]:
t[t['Hotel_Name'] == 'Hotel Regina']

,Hotel_Name,Country_Name,Positive_Review,Review_Total_Positive_Word_Counts,Reviewer_Score
741,Hotel Regina,Austria,Many breakfast choices with table service for...,5507,8.390845
742,Hotel Regina,Italy,I stayed entire month at the hotel every sing...,1095,8.420000
743,Hotel Regina,Spain,Beautiful hotel Very helpful staff GREAT loca...,9881,8.538814


In [19]:
p[p['Hotel_Name'] == 'Hotel Regina']

,Hotel_Name,Tags
741,Hotel Regina,Leisure Stayed 1 night Couple Austria Argentina


In [20]:

t = t.drop(t[(t.Hotel_Name =='Hotel Regina') & (t.Country_Name == 'Italy')].index)
t = t.drop(t[(t.Hotel_Name =='Hotel Regina') & (t.Country_Name == 'Spain')].index)

In [21]:
t = t.sort_values(by='Hotel_Name', ascending=True)
p = p.sort_values(by='Hotel_Name', ascending=True)
p = p.reset_index(drop=True)
t = t.reset_index(drop=True)

In [22]:
a = t
b = p

In [23]:
e = t

In [24]:
#t['Negative_Review'] = t['Negative_Review'].apply(lambda x: x.replace("No Negative", "").replace("No Positive", ""))

t['Positive_Review'] = t['Positive_Review'].apply(lambda x: x.replace("No Negative", "").replace("No Positive", ""))

def review_cleaned(text_data):
    review_text_cleaned1 = text_data
    split_sentence = re.sub("[^a-zA-Z]", " ",text_data)
    tokens = nltk.word_tokenize(split_sentence)
    lower_case_words = [p.lower() for p in tokens]
    filtered_value = list(filter(lambda p: p not in stop_words_english, lower_case_words))
    lemma_word = [wordnet_lemmatizer.lemmatize(e) for e in filtered_value] 
    removed_1_text = [e for e in lemma_word if len(e) > 1]
    review_text_cleaned1 = ' '.join(removed_1_text)
    t = Counter(ngrams(review_text_cleaned1.split(), 2)).most_common(15)
    res = str(t).strip('[]') 
    res1 = re.sub("[^a-zA-Z]", " ",res) 
    res2 = re.sub(' +', ' ', res1)
    return(res2)

# clean text data
import re,nltk
from collections import Counter
from nltk.util import ngrams 
import pandas as pd
stop_words_english = set(stopwords.words('english'))
wordnet_lemmatizer = WordNetLemmatizer()
#t['Negative_Summarized'] = t['Negative_Review'].apply(lambda x: review_cleaned(x))
t['Positive_Summarized'] = t['Positive_Review'].apply(lambda x: review_cleaned(x))

In [25]:
t.head(1)

,Hotel_Name,Country_Name,Positive_Review,Review_Total_Positive_Word_Counts,Reviewer_Score,Positive_Summarized
0,11 Cadogan Gardens,United Kingdom,We were particularly impressed by the very wa...,3176,8.845283,helpful friendly friendly staff friendly help...


In [26]:
t['Positive_Summarized'] = t['Positive_Summarized'] + p['Tags']
t = t.dropna()
t = t.reset_index(drop=True)

In [27]:
t = t.drop(['Positive_Review'], axis=1)

In [28]:
t[['Hotel_Name','Positive_Summarized','Country_Name']].head(5)

,Hotel_Name,Positive_Summarized,Country_Name
0,11 Cadogan Gardens,helpful friendly friendly staff friendly help...,United Kingdom
1,1K Hotel,great location helpful staff friendly staff g...,France
2,25hours Hotel beim MuseumsQuartier,rooftop bar great location friendly staff roo...,Austria
3,41,executive lounge made feel attention detail n...,United Kingdom
4,45 Park Lane Dorchester Collection,absolutely wonderful every thing everything a...,United Kingdom


In [30]:
t.to_csv('RecommenderData8.csv',index=False)

In [31]:
import numpy as np
import re, math
import json
from collections import Counter
import pandas as pd
import operator

In [32]:
def find_cosine_similarity(t1, t2):
    #get words first
    t1 = re.sub("[^a-zA-Z]",  
                          " ",          
                          str(t1))
    first_text = re.compile(r"[\w']+").findall(t1)
    second_text = re.compile(r"[\w']+").findall(t2)

    vector_first = Counter(first_text)
    vector_second = Counter(second_text)

    frequent = set(vector_first.keys()).intersection(set(vector_second.keys()))

    dot_pdt = 0.0

    for j in frequent:
        dot_pdt += vector_first[j] * vector_second[j]

    sq_sum_vector_first = 0.0
    sq_sum_vector_second = 0.0

    for i in vector_first.keys():
        sq_sum_vector_first += vector_first[i]**2

    for i in vector_second.keys():
        sq_sum_vector_second += vector_second[i]**2

    mag = math.sqrt(sq_sum_vector_first) * math.sqrt(sq_sum_vector_second)

    if not mag:
        return 0.0
    else:
        return float(dot_pdt) / mag

In [37]:
def recommendations_engine(keywords):

    dataFrame = pd.read_csv('C:\\Users\\himan\\RecommenderData8.csv')

    score_dict = {}

    for index, row in dataFrame.iterrows():
        score_dict[index] = find_cosine_similarity(row['Positive_Summarized'], keywords)

    #Countries are sorted based on the score
    sorted_scores = sorted(score_dict.items(), key=operator.itemgetter(1), reverse=True)

    counter = 0

    #Build a blank dataframe
    resultant = pd.DataFrame(columns=('Hotel_Name', 'Review_Total_Positive_Word_Counts', 'Positive_Summarized', 'Score'))

    #Top 5 sorted countries
    for i in sorted_scores:
        resultant = resultant.append({'Hotel_Name': dataFrame.iloc[i[0]]['Hotel_Name'], 'Review_Total_Positive_Word_Counts': dataFrame.iloc[i[0]]['Review_Total_Positive_Word_Counts'], 'Positive_Summarized': dataFrame.iloc[i[0]]['Positive_Summarized'], 'Score': i[1]}, ignore_index=True)
        counter += 1

        if counter>4:
            break
    #Dump to JSON
    json_result = json.dumps(resultant.to_dict('records'))
    return json_result

In [38]:
import json

more_match = "good staff clean room great location metro near gym beach view, big room breakfast United Kingdom"

def find_recommendations(keywords):
    result = recommendations_engine(keywords)
    return result

def find_top_5_hotel_names(json_string):
    list = json.loads(json_string)
    result = []
    max = len(list)
    i = 0
    while i < max:
        result.append((list[i]['Hotel_Name'], list[i]['Score']))
        i += 1

    return result

print("Top 5 Hotels with max text matching based on Reviews given")
top_5_max_text_match = find_recommendations(more_match)
df_more_match = pd.DataFrame((find_top_5_hotel_names(top_5_max_text_match)),columns=['Hotel_Name', 'Score'])
df_more_match

Top 5 Hotels with max text matching based on Reviews given


,Hotel_Name,Score
0,NH Wien City,0.676802
1,NH Collection Wien Zentrum,0.647150
2,Ilunion Barcelona,0.644658
3,Novotel Suites Paris Nord 18 me,0.642857
4,Staunton Hotel B B,0.642416


In [39]:
more_match = "great location helpful staff close metro room size metro station clean room friendly helpful location excellent value money Business Stayed 1 night Group France Australia"
#more_match = "staff friendly room clean"
#more_match = "Leisure Stayed 6 night Couple United Kingdom"

def find_recommendations(keywords):
    result = recommendations_engine(keywords)
    return result

def find_top_5_hotel_names(json_string):
    list = json.loads(json_string)
    result = []
    max = len(list)
    i = 0
    while i < max:
        result.append((list[i]['Hotel_Name'], list[i]['Score']))
        i += 1

    return result

print("Top 5 Hotels with max text matching based on Reviews given")
top_5_max_text_match = find_recommendations(more_match)
df_more_match = pd.DataFrame((find_top_5_hotel_names(top_5_max_text_match)),columns=['Hotel_Name', 'Score'])
df_more_match

Top 5 Hotels with max text matching based on Reviews given


,Hotel_Name,Score
0,1K Hotel,0.873698
1,H tel Aiglon Esprit de France,0.758787
2,Amp re,0.745193
3,Best Western Premier Marais Grands Boulevards,0.742270
4,Forest Hill Paris la Villette,0.738851


In [40]:
no_match = "bad quality no space untidy room high price America Brazil"
print("Top 5 Hotels with text matching based on Reviews and Other data given")
top_5_no_text_match = find_recommendations(no_match)
df_no_match = pd.DataFrame((find_top_5_hotel_names(top_5_no_text_match)),columns=['Hotel_Name', 'Score'])
df_no_match

Top 5 Hotels with text matching based on Reviews and Other data given


,Hotel_Name,Score
0,Milan Suite Hotel,0.262000
1,Klima Hotel Milano Fiere,0.250640
2,Hotel Da Vinci,0.244567
3,Crowne Plaza Barcelona Fira Center,0.243432
4,Arion Cityhotel Vienna und Appartements,0.241523


In [41]:
t['Positive_Summarized'][5]

' comfortable bed clean modern staff friendly room clean great location friendly helpful minute walk comfy bed value money friendly staff well equipped clean comfortable helpful staff tea coffee tube station Leisure Stayed 10 night Group United Kingdom  Australia '

In [42]:
no_match = "ticket cheap fight ontime cabin crew supportive pilot trained"
print("Top 5 Hotels with text matching based on Reviews and Other data given")
top_5_no_text_match = find_recommendations(no_match)
df_no_match = pd.DataFrame((find_top_5_hotel_names(top_5_no_text_match)),columns=['Hotel_Name', 'Score'])
df_no_match

Top 5 Hotels with text matching based on Reviews and Other data given


,Hotel_Name,Score
0,IntercityHotel Wien,0.03849
1,11 Cadogan Gardens,0.00000
2,1K Hotel,0.00000
3,25hours Hotel beim MuseumsQuartier,0.00000
4,41,0.00000


In [43]:
all_match = "good breakfast helpful staff location excellent room spacious clean nicely business center comfortable nice coffee machine staff nice bed comfortable friendly always helpful room Business Stayed 1 night Solo Spain  Andorra"
print("Top 5 Hotels with text matching based on Reviews and Other data given")
top_5_all_text_match = find_recommendations(all_match)
df_all_match = pd.DataFrame((find_top_5_hotel_names(top_5_all_text_match)),columns=['Hotel_Name', 'Score'])
df_all_match

Top 5 Hotels with text matching based on Reviews and Other data given


,Hotel_Name,Score
0,AC Hotel Diagonal L Illa a Marriott Lifestyle ...,0.907742
1,NH Amsterdam Zuid,0.748983
2,Eurostars Angli,0.746653
3,Le Mathurin Hotel Spa,0.731150
4,Bianca Maria Palace Hotel,0.724148


In [44]:
t['Positive_Summarized'][10]

' good breakfast helpful staff location excellent room spacious friendly staff clean nicely business center comfortable nice coffee machine staff nice bed comfortable comfortable staff staff friendly friendly always helpful room Business Stayed 1 night Solo Spain  Andorra '

In [45]:
less_match = "friendly staff room clean spacious room good Wifi good breakfast Business Stayed 2 night Couple Austria"
print("Top 5 Hotels with text matching based on Reviews and Other data given")
top_5_less_text_match = find_recommendations(less_match)
df_less_match = pd.DataFrame((find_top_5_hotel_names(top_5_less_text_match)),columns=['Hotel_Name', 'Score'])
df_less_match

Top 5 Hotels with text matching based on Reviews and Other data given


,Hotel_Name,Score
0,Austria Trend Hotel Lassalle Wien,0.774597
1,Exe Vienna,0.745394
2,Hotel Kavalier,0.734130
3,Arion Cityhotel Vienna und Appartements,0.731925
4,Idea Hotel Milano San Siro,0.729574


In [47]:
l_match = "good breakfast helpful staff location excellent room spacious clean nicely business center comfortable nice coffee machine staff nice bed comfortable friendly always helpful room Business Stayed 1 night Solo Spain Andorra"
print("Top 5 Hotels with text matching based on Reviews and Other data given")
top_5_l_text_match = find_recommendations(l_match)
df_l_match = pd.DataFrame((find_top_5_hotel_names(top_5_l_text_match)),columns=['Hotel_Name', 'Score'])
df_l_match

Top 5 Hotels with text matching based on Reviews and Other data given


,Hotel_Name,Score
0,AC Hotel Diagonal L Illa a Marriott Lifestyle ...,0.907742
1,NH Amsterdam Zuid,0.748983
2,Eurostars Angli,0.746653
3,Le Mathurin Hotel Spa,0.731150
4,Bianca Maria Palace Hotel,0.724148
